# DATA COLLECTION & CLEANSING
------------------------------------------------

# Dependencies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from pprint import pprint
from config import api_key
import numpy as np
import os
import shutil

# Data Collection

In [2]:
#The College Scorecard API is a GET API that lives at http://api.data.gov/ed/collegescorecard/
#The endpoint for querying all data is /v1/schools
base_url = "https://api.data.gov/ed/collegescorecard/v1/schools?"
base_url = f"{base_url}api_key={api_key}&fields="
base_url

'https://api.data.gov/ed/collegescorecard/v1/schools?api_key=qU5kxpGGsRBz1SbYzlvidBgnbKC1yQ0lX30EU6Mo&fields='

In [3]:
# List of all the search conditions
parameters = ["&school.degrees_awarded.predominant=3",
             "&school.institutional_characteristics.level=1",
             "&school.operating=1"
             ]

# Appending all the conditions values to construct the conditionss_url
parameters_url = ""
for parameter in parameters:
    parameters_url = parameters_url + parameter
parameters_url

'&school.degrees_awarded.predominant=3&school.institutional_characteristics.level=1&school.operating=1'

In [4]:
# Dictionary all the desired fields
year = "latest"
fields = {
          # School Category
          "School Name": "school.name",
          "School ID": "id",
          "School State": "school.state",
          "School Ownership": "school.ownership",
          "Full-time Faculty Rate (%)": "school.ft_faculty_rate",
          "Faculty's average salary per month": "school.faculty_salary",
          # Student Category
          "Student Enrollment Size": year + ".student.size",
          "Student Enrollment All": year + ".student.enrollment.all",
          "Male Students (%)": year + ".student.demographics.men",
          "Female Students (%)": year + ".student.demographics.women",
          "Retention Rate 4Yr (%)": year + ".student.retention_rate.four_year.full_time",
          #Cost Category
          "Attendance Cost per Academic Year": year + ".cost.attendance.academic_year",
          # Completion Category
          "150% Completion Rate at 4Yr (%)": year + ".completion.completion_rate_4yr_150nt",
          # Admissions Category 
          "Admission Rate (%)": year + ".admissions.admission_rate.overall",
          "SAT Average Overall": year + ".admissions.sat_scores.average.overall",
          "SAT 75th Percentile Critical Math": year + ".admissions.sat_scores.75th_percentile.math",
          "SAT 75th Percentile Critical Reading": year + ".admissions.sat_scores.75th_percentile.critical_reading",
          "SAT 75th Percentile Critical Writing": year + ".admissions.sat_scores.75th_percentile.writing",
          ## Earnings Category
          # 6 Years after Enrollment:
          "Mean Earnings (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.mean_earnings",
          "Mean Male Earnings (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.male_students",
          "Mean Female Earnings (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.female_students",
          "Std. Deviation Earning (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.std_dev",
          "Percent of Students Earning >$25K (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.percent_greater_than_25000",
          "Low Income Students (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.income.lowest_tercile",
          "Medium Income Students (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.income.middle_tercile",
          "High Income Students (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.income.highest_tercile",
          "Mean Earnings Low (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.lowest_tercile",
          "Mean Earnings Medium (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.middle_tercile",
          "Mean Earnings High (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.highest_tercile",
          # 10 Years after Enrollment:
          "Mean Earnings (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.mean_earnings",
          "Mean Male Earnings (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.male_students",
          "Mean Female Earnings (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.female_students",
          "Std. Deviation Earning (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.std_dev",
          "Percent of Students Earning >$25K (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.percent_greater_than_25000",
          "Low Income Students (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.income.lowest_tercile",
          "Medium Income Students (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.income.middle_tercile",
          "High Income Students (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.income.highest_tercile",
          "Mean Earnings Low (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.lowest_tercile",
          "Mean Earnings Medium (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.middle_tercile",
          "Mean Earnings High (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.highest_tercile"
         }

# Appending all the fields values to construct the fields_url
fields_url = ""
for key, val in fields.items():
    fields_url = fields_url + val + ","

# To remove the extra "," at the end of fields_url
fields_url = fields_url[:-1]
fields_url

'school.name,id,school.state,school.ownership,school.ft_faculty_rate,school.faculty_salary,latest.student.size,latest.student.enrollment.all,latest.student.demographics.men,latest.student.demographics.women,latest.student.retention_rate.four_year.full_time,latest.cost.attendance.academic_year,latest.completion.completion_rate_4yr_150nt,latest.admissions.admission_rate.overall,latest.admissions.sat_scores.average.overall,latest.admissions.sat_scores.75th_percentile.math,latest.admissions.sat_scores.75th_percentile.critical_reading,latest.admissions.sat_scores.75th_percentile.writing,latest.earnings.6_yrs_after_entry.working_not_enrolled.mean_earnings,latest.earnings.6_yrs_after_entry.mean_earnings.male_students,latest.earnings.6_yrs_after_entry.mean_earnings.female_students,latest.earnings.6_yrs_after_entry.working_not_enrolled.std_dev,latest.earnings.6_yrs_after_entry.percent_greater_than_25000,latest.earnings.6_yrs_after_entry.working_not_enrolled.income.lowest_tercile,latest.earnings

In [5]:
# Getting number of records returned to set the max page number
query_url = f"{base_url}{fields_url}{parameters_url}&page=0"
response = requests.get(query_url).json()
max_page_num = response["metadata"]["total"]//100 + 1
max_page_num

21

In [6]:
# Constructing the dataframe from the API request response

#Initializing variables
school_df = []
per_page = 100

for page_num in range(0,max_page_num):
    query_url = f"{base_url}{fields_url}{parameters_url}&page={page_num}&_per_page={per_page}"
    response = requests.get(query_url).json()
    
    for x in range(len(response["results"])):
        result_row = {}
        
        for key, val in fields.items(): 
            try:
                result_row[key] = response["results"][x][val]
            except KeyError:
                print(f"{key} key not found")
                
        school_df.append(result_row)

        
school_df = pd.DataFrame(school_df)
school_df.head()

,150% Completion Rate at 4Yr (%),Admission Rate (%),Attendance Cost per Academic Year,Faculty's average salary per month,Female Students (%),Full-time Faculty Rate (%),High Income Students (10 Yrs after Entry),High Income Students (6 Yrs after Entry),Low Income Students (10 Yrs after Entry),Low Income Students (6 Yrs after Entry),...,SAT 75th Percentile Critical Writing,SAT Average Overall,School ID,School Name,School Ownership,School State,Std. Deviation Earning (10 Yrs after Entry),Std. Deviation Earning (6 Yrs after Entry),Student Enrollment All,Student Enrollment Size
0,0.3000,NaN,29544.0,6896.0,0.5925,0.0782,1854.0,2383.0,3049.0,7701.0,...,NaN,NaN,449038,Strayer University-Florida,3,FL,39800.0,29300.0,NaN,1698.0
1,NaN,NaN,29544.0,7980.0,0.7033,NaN,1854.0,2383.0,3049.0,7701.0,...,NaN,NaN,450298,Strayer University-Delaware,3,DE,39800.0,29300.0,NaN,246.0
2,0.5833,0.9839,32156.0,3750.0,0.5904,0.2857,NaN,NaN,NaN,NaN,...,NaN,NaN,182917,Northeast Catholic College,2,NH,NaN,NaN,NaN,83.0
3,0.3256,0.6817,14587.0,8807.0,0.6832,1.0000,2937.0,3234.0,2499.0,2961.0,...,NaN,917.0,484905,University of North Texas at Dallas,1,TX,35100.0,25800.0,NaN,2257.0
4,NaN,1.0000,40811.0,6857.0,0.8660,0.2031,NaN,566.0,41.0,586.0,...,NaN,NaN,485272,West Coast University-Miami,3,FL,42100.0,35500.0,NaN,485.0


# Data Cleansing

In [7]:
# Updating School Ownership 1: "Public", 2: "Private NonProfit", 3: "Private ForProfit"}
school_df.loc[school_df["School Ownership"] == 1, "School Ownership"] = "Public"
school_df.loc[school_df["School Ownership"] == 2, "School Ownership"] = "Private NonProfit"
school_df.loc[school_df["School Ownership"] == 3, "School Ownership"] = "Private ForProfit"

In [8]:
# Formatting the Percentage columns
school_df["Admission Rate (%)"] = school_df["Admission Rate (%)"] * 100
school_df["150% Completion Rate at 4Yr (%)"] = school_df["150% Completion Rate at 4Yr (%)"] * 100
school_df["Retention Rate 4Yr (%)"] = school_df["Retention Rate 4Yr (%)"] * 100
school_df["Full-time Faculty Rate (%)"] = school_df["Full-time Faculty Rate (%)"] * 100
school_df["Percent of Students Earning >$25K (6 Yrs after Entry)"] = school_df["Percent of Students Earning >$25K (6 Yrs after Entry)"] * 100
school_df["Percent of Students Earning >$25K (10 Yrs after Entry)"] = school_df["Percent of Students Earning >$25K (10 Yrs after Entry)"] * 100
school_df["Male Students (%)"] = school_df["Male Students (%)"] * 100
school_df["Female Students (%)"] = school_df["Female Students (%)"] * 100

pd.options.display.float_format = '{:.2f}'.format

In [9]:
# Creating Directories & Saving Dataframe in CSV file
shutil.rmtree("Resources", ignore_errors = True)
shutil.rmtree("Plots", ignore_errors = True)

path_plot_ownadmission = os.path.join("Resources", "Plots", "Ownership_Admission")
os.makedirs(path_plot_ownadmission, exist_ok=True)

path_plot_select = os.path.join("Resources", "Plots", "Selectiveness")
os.makedirs(path_plot_select, exist_ok=True)

path_plot_compretention = os.path.join("Resources", "Plots", "Completion_Retention")
os.makedirs(path_plot_compretention, exist_ok=True)

path_plot_faculty = os.path.join("Resources", "Plots", "Faculty")
os.makedirs(path_plot_faculty, exist_ok=True)

path_csv_ownadmission = os.path.join("Resources", "CSV", "Ownership_Admission")
os.makedirs(path_csv_ownadmission, exist_ok=True)

path_csv_select = os.path.join("Resources", "CSV", "Selectiveness")
os.makedirs(path_csv_select, exist_ok=True)

path_csv_compretention = os.path.join("Resources", "CSV", "Completion_Retention")
os.makedirs(path_csv_compretention, exist_ok=True)

path_csv_faculty = os.path.join("Resources", "CSV", "Faculty")
os.makedirs(path_csv_faculty, exist_ok=True)

path_csv_general = os.path.join("Resources", "CSV", "General")
os.makedirs(path_csv_general, exist_ok=True)

# Saving Dataframe in CSV file
file_name = "Schools_DF.csv"
school_df.to_csv(os.path.join(path_csv_general, file_name), index=False)

# Creating sub dataframes

In [10]:
# Creating sub dataframe to Analyze School Ownership and Admission Rates
school_ownership_df = school_df[["School ID","School Name","School State","School Ownership",
                           "Admission Rate (%)", "Mean Earnings (10 Yrs after Entry)", "Mean Earnings (6 Yrs after Entry)"
                                 ]].copy()
clean_school_ownership_df = school_ownership_df.dropna()
file_name = "School_Ownership_DF.csv"
clean_school_ownership_df.to_csv(os.path.join(path_csv_ownadmission, file_name), index=False)

In [11]:
# Creating sub dataframe to Analyze School's Selecteviness (SAT Scores) to School's Performance
selectiveness_df = school_df[["School Name", "School ID", "Admission Rate (%)",
                             "SAT Average Overall", "Mean Earnings (6 Yrs after Entry)",
                             "Retention Rate 4Yr (%)", "150% Completion Rate at 4Yr (%)",
                             "Mean Earnings (10 Yrs after Entry)", "Attendance Cost per Academic Year",
                             "School Ownership", "School State"]].copy()
selectiveness_df.dropna(inplace=True)
file_name = "Selectiveness_DF.csv"
selectiveness_df.to_csv(os.path.join(path_csv_select, file_name), index=False)

In [12]:
# Creating sub dataframe to Analyze School's Completion rate and Retention Rate
selectdata_df = school_df[["School Name","School State","School Ownership",
                         "Mean Earnings (6 Yrs after Entry)",
                        "Std. Deviation Earning (6 Yrs after Entry)",
                         "Mean Earnings (10 Yrs after Entry)",
                        "Std. Deviation Earning (10 Yrs after Entry)",
                         "150% Completion Rate at 4Yr (%)",
                         "Retention Rate 4Yr (%)"
                        ]].copy()
selectdata_df.dropna(how='any', inplace=True)
file_name = "Completion_Retention_DF.csv"
selectdata_df.to_csv(os.path.join(path_csv_compretention, file_name), index=False)

In [13]:
# Creating sub dataframe to Analyze Full-time Faculty Rate and Faculty's average salary per month to Mean Earnings
faculty_df = school_df[["School ID", "School Name", "School State", "Full-time Faculty Rate (%)", 
                        "Faculty's average salary per month", "Mean Earnings (6 Yrs after Entry)", "Mean Earnings (10 Yrs after Entry)"]].copy()
faculty_df.dropna(inplace=True)
file_name = "Faculty_DF.csv"
faculty_df.to_csv(os.path.join(path_csv_faculty, file_name), index=False)

# Aggregations

In [14]:
# Aggregation for summary table (Admission information)

# Admission Rate
admission_mean = clean_school_ownership_df["Admission Rate (%)"].mean()
admission_max = clean_school_ownership_df["Admission Rate (%)"].max()
admission_min = clean_school_ownership_df["Admission Rate (%)"].min()

# Mean Earnings (6 yrs after entry)
admission_earnings_6yr_mean = clean_school_ownership_df["Mean Earnings (6 Yrs after Entry)"].mean()
admission_earnings_6yr_max = clean_school_ownership_df["Mean Earnings (6 Yrs after Entry)"].max()
admission_earnings_6yr_min = clean_school_ownership_df["Mean Earnings (6 Yrs after Entry)"].min()

# Mean Earnings (10 yrs after entry)
admission_earnings_10yr_mean = clean_school_ownership_df["Mean Earnings (10 Yrs after Entry)"].mean()
admission_earnings_10yr_max = clean_school_ownership_df["Mean Earnings (10 Yrs after Entry)"].max()
admission_earnings_10yr_min = clean_school_ownership_df["Mean Earnings (10 Yrs after Entry)"].min()

# Create lists to hold the calculations
admission_list = [admission_max,admission_min,admission_mean]
admission_earnings_6yr_list = [admission_earnings_6yr_max,admission_earnings_6yr_min,admission_earnings_6yr_mean]
admission_earnings_10yr_list = [admission_earnings_10yr_max,admission_earnings_10yr_min,admission_earnings_10yr_mean]

In [15]:
# Aggregation for summary table (Faculty information)

# Faculty's average salary per month
faculty_avg_salary_max = faculty_df["Faculty's average salary per month"].max()
faculty_avg_salary_min = faculty_df["Faculty's average salary per month"].min()
faculty_avg_salary_mean = faculty_df["Faculty's average salary per month"].mean()

# Full-time faculty rate
faculty_ft_rate_max = faculty_df["Full-time Faculty Rate (%)"].max()
faculty_ft_rate_min = faculty_df["Full-time Faculty Rate (%)"].min()
faculty_ft_rate_mean = faculty_df["Full-time Faculty Rate (%)"].mean()

# Mean Earnings (6 yrs after entry)
faculty_mean_earnings_6_yrs_max = faculty_df["Mean Earnings (6 Yrs after Entry)"].max()
faculty_mean_earnings_6_yrs_min = faculty_df["Mean Earnings (6 Yrs after Entry)"].min()
faculty_mean_earnings_6_yrs_mean = faculty_df["Mean Earnings (6 Yrs after Entry)"].mean()

# Mean Earnings (10 yrs after entry)
faculty_mean_earnings_10_yrs_max = faculty_df["Mean Earnings (10 Yrs after Entry)"].max()
faculty_mean_earnings_10_yrs_min = faculty_df["Mean Earnings (10 Yrs after Entry)"].min()
faculty_mean_earnings_10_yrs_mean = faculty_df["Mean Earnings (10 Yrs after Entry)"].mean()

# Create lists to hold the calculations
faculty_ft_rate_list = [faculty_ft_rate_max, faculty_ft_rate_min, faculty_ft_rate_mean]
faculty_avg_salary_list = [faculty_avg_salary_max, faculty_avg_salary_min, faculty_avg_salary_mean]
faculty_mean_earnings_6_yrs_list = [faculty_mean_earnings_6_yrs_max, faculty_mean_earnings_6_yrs_min, faculty_mean_earnings_6_yrs_mean]
faculty_mean_earnings_10_yrs_list = [faculty_mean_earnings_10_yrs_max, faculty_mean_earnings_10_yrs_min, faculty_mean_earnings_10_yrs_mean]

In [16]:
# Aggregation for summary table (SAT information)

# SAT scores
SAT_score_max = selectiveness_df["SAT Average Overall"].max()
SAT_score_min = selectiveness_df["SAT Average Overall"].min()
SAT_score_mean = selectiveness_df["SAT Average Overall"].mean()

# Mean Earnings (6 yrs after entry)
SAT_mean_earnings_6_yrs_max = selectiveness_df["Mean Earnings (6 Yrs after Entry)"].max()
SAT_mean_earnings_6_yrs_min = selectiveness_df["Mean Earnings (6 Yrs after Entry)"].min()
SAT_mean_earnings_6_yrs_mean = selectiveness_df["Mean Earnings (6 Yrs after Entry)"].mean()

# Mean Earnings (10 yrs after entry)
SAT_mean_earnings_10_yrs_max = selectiveness_df["Mean Earnings (10 Yrs after Entry)"].max()
SAT_mean_earnings_10_yrs_min = selectiveness_df["Mean Earnings (10 Yrs after Entry)"].min()
SAT_mean_earnings_10_yrs_mean = selectiveness_df["Mean Earnings (10 Yrs after Entry)"].mean()

# Create lists to hold the calculations
SAT_score_list = [SAT_score_max, SAT_score_min, SAT_score_mean]
SAT_mean_earnings_6_yrs_list = [SAT_mean_earnings_6_yrs_max, SAT_mean_earnings_6_yrs_min, SAT_mean_earnings_6_yrs_mean]
SAT_mean_earnings_10_yrs_list = [SAT_mean_earnings_10_yrs_max, SAT_mean_earnings_10_yrs_min, SAT_mean_earnings_10_yrs_mean]

In [17]:
# Data cleansing: Binning completion rate groups
completion_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 101]
completion_group_names = ["0-10", "10-20", "20-30", "30-40", "40-50", "50-60", "60-70", "70-80", "80-90", "90-100"]
selectdata_df["Completion Rate Group"] = pd.cut(selectdata_df["150% Completion Rate at 4Yr (%)"], 
                                  completion_bins, labels=completion_group_names)

In [18]:
# Data cleansing: Binning retention rate groups
retention_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 101]
retention_group_names = ["0-10", "10-20", "20-30", "30-40", "40-50", "50-60", "60-70", "70-80", "80-90", "90-100"]
selectdata_df["Retention Rate Group"] = pd.cut(selectdata_df["Retention Rate 4Yr (%)"], 
                                  retention_bins, labels=retention_group_names)

In [19]:
# Data cleansing: Completion rate data (6yrs)
selectdata_completion_df=selectdata_df[["Completion Rate Group","Mean Earnings (6 Yrs after Entry)"]].set_index("Completion Rate Group")

completion_0_df=selectdata_completion_df.loc["0-10"].reset_index(drop=True)
completion_10_df=selectdata_completion_df.loc["10-20"].reset_index(drop=True)
completion_20_df=selectdata_completion_df.loc["20-30"].reset_index(drop=True)
completion_30_df=selectdata_completion_df.loc["30-40"].reset_index(drop=True)
completion_40_df=selectdata_completion_df.loc["40-50"].reset_index(drop=True)
completion_50_df=selectdata_completion_df.loc["50-60"].reset_index(drop=True)
completion_60_df=selectdata_completion_df.loc["60-70"].reset_index(drop=True)
completion_70_df=selectdata_completion_df.loc["70-80"].reset_index(drop=True)
completion_80_df=selectdata_completion_df.loc["80-90"].reset_index(drop=True)
completion_90_df=selectdata_completion_df.loc["90-100"].reset_index(drop=True)
completion_6yrs = pd.concat([completion_0_df,
                         completion_10_df,
                         completion_20_df,
                         completion_30_df,
                         completion_40_df,
                         completion_50_df,
                         completion_60_df,
                         completion_70_df,
                         completion_80_df,
                         completion_90_df,
                        ],axis=1)
completion_6yrs.columns=["0-10", "10-20", "20-30", "30-40", "40-50", "50-60", "60-70", "70-80", "80-90", "90-100"]

# Data cleansing: Completion rate data (10yrs)
selectdata_completion_df=selectdata_df[["Completion Rate Group","Mean Earnings (10 Yrs after Entry)"]].set_index("Completion Rate Group")

completion_0_df=selectdata_completion_df.loc["0-10"].reset_index(drop=True)
completion_10_df=selectdata_completion_df.loc["10-20"].reset_index(drop=True)
completion_20_df=selectdata_completion_df.loc["20-30"].reset_index(drop=True)
completion_30_df=selectdata_completion_df.loc["30-40"].reset_index(drop=True)
completion_40_df=selectdata_completion_df.loc["40-50"].reset_index(drop=True)
completion_50_df=selectdata_completion_df.loc["50-60"].reset_index(drop=True)
completion_60_df=selectdata_completion_df.loc["60-70"].reset_index(drop=True)
completion_70_df=selectdata_completion_df.loc["70-80"].reset_index(drop=True)
completion_80_df=selectdata_completion_df.loc["80-90"].reset_index(drop=True)
completion_90_df=selectdata_completion_df.loc["90-100"].reset_index(drop=True)
completion_10yrs = pd.concat([completion_0_df,
                         completion_10_df,
                         completion_20_df,
                         completion_30_df,
                         completion_40_df,
                         completion_50_df,
                         completion_60_df,
                         completion_70_df,
                         completion_80_df,
                         completion_90_df,
                        ],axis=1)
completion_10yrs.columns=["0-10", "10-20", "20-30", "30-40", "40-50", "50-60", "60-70", "70-80", "80-90", "90-100"]

In [20]:
# Data cleansing: Retention rate data (6yrs)
selectdata_retention_df=selectdata_df[["Retention Rate Group","Mean Earnings (6 Yrs after Entry)"]].set_index("Retention Rate Group")

retention_0_df=selectdata_retention_df.loc["0-10"].reset_index(drop=True)
retention_10_df=selectdata_retention_df.loc["10-20"].reset_index(drop=True)
retention_20_df=selectdata_retention_df.loc["20-30"].reset_index(drop=True)
retention_30_df=selectdata_retention_df.loc["30-40"].reset_index(drop=True)
retention_40_df=selectdata_retention_df.loc["40-50"].reset_index(drop=True)
retention_50_df=selectdata_retention_df.loc["50-60"].reset_index(drop=True)
retention_60_df=selectdata_retention_df.loc["60-70"].reset_index(drop=True)
retention_70_df=selectdata_retention_df.loc["70-80"].reset_index(drop=True)
retention_80_df=selectdata_retention_df.loc["80-90"].reset_index(drop=True)
retention_90_df=selectdata_retention_df.loc["90-100"].reset_index(drop=True)
retention_6yrs = pd.concat([retention_0_df,
                         retention_10_df,
                         retention_20_df,
                         retention_30_df,
                         retention_40_df,
                         retention_50_df,
                         retention_60_df,
                         retention_70_df,
                         retention_80_df,
                         retention_90_df,
                        ],axis=1)
retention_6yrs.columns=["0-10", "10-20", "20-30", "30-40", "40-50", "50-60", "60-70", "70-80", "80-90", "90-100"]

# Data cleansing: Retention rate data (10yrs)
selectdata_retention_df=selectdata_df[["Retention Rate Group","Mean Earnings (10 Yrs after Entry)"]].set_index("Retention Rate Group")

retention_0_df=selectdata_retention_df.loc["0-10"].reset_index(drop=True)
retention_10_df=selectdata_retention_df.loc["10-20"].reset_index(drop=True)
retention_20_df=selectdata_retention_df.loc["20-30"].reset_index(drop=True)
retention_30_df=selectdata_retention_df.loc["30-40"].reset_index(drop=True)
retention_40_df=selectdata_retention_df.loc["40-50"].reset_index(drop=True)
retention_50_df=selectdata_retention_df.loc["50-60"].reset_index(drop=True)
retention_60_df=selectdata_retention_df.loc["60-70"].reset_index(drop=True)
retention_70_df=selectdata_retention_df.loc["70-80"].reset_index(drop=True)
retention_80_df=selectdata_retention_df.loc["80-90"].reset_index(drop=True)
retention_90_df=selectdata_retention_df.loc["90-100"].reset_index(drop=True)
retention_10yrs = pd.concat([retention_0_df,
                         retention_10_df,
                         retention_20_df,
                         retention_30_df,
                         retention_40_df,
                         retention_50_df,
                         retention_60_df,
                         retention_70_df,
                         retention_80_df,
                         retention_90_df,
                        ],axis=1)
retention_10yrs.columns=["0-10", "10-20", "20-30", "30-40", "40-50", "50-60", "60-70", "70-80", "80-90", "90-100"]

In [21]:
print("Completed Retrieving and Initial Cleansing and Aggregation of API Dataset.")

Completed Retrieving and Initial Cleansing and Aggregation of API Dataset.
